In [ ]:
# basics
import os
import numpy as np
import pandas as pd

#plotting
import matplotlib
import matplotlib.pyplot as plt
from adjustText import adjust_text


# warnings and debugging
import importlib
import ipdb; #ipdb.set_trace()
import logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings("once")  # Ensures warnings appear on every cell run
#warnings.simplefilter("once")
#warnings.simplefilter("error") #handle warnings as errors (used for DepreactionWarnings)
#warnings.simplefilter("default")

#extras
from tqdm import tqdm
from collections import OrderedDict
import joblib

# modelbase
import modelbase
from modelbase.ode import Simulator
from scipy import integrate

print(importlib.metadata.version("modelbase"))

# working directory
script_dir = globals().get("script_dir", os.getcwd()) # if script_dir exists, use it, otherwise use cwd
print(script_dir)
os.chdir(f"{script_dir}/..")
print(os.getcwd())


colours = OrderedDict({"blue": "#377EB8",   "orange": "#FF7F00",
                        "green": "#1A4D2E", "pink": "#E6518C",
                        "purple": "#984EA3","red": "#E41A1C",
                        "brown": "#A65628", "grey": "#999999"})
colors = list(colours.values())


# load photosynthesis model
import models
importlib.reload(models)
from models import get_model

m = get_model()
